# Data Augumentation

In [2]:
from datasets import load_dataset, Dataset, concatenate_datasets
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from pprint import pprint
import pandas as pd
from tqdm.notebook import tqdm
from huggingface_hub import login

In [2]:
login()

In [3]:
sample_ds=load_dataset("s-nlp/paradetox", split="train")
sample_ds

Dataset({
    features: ['en_toxic_comment', 'en_neutral_comment'],
    num_rows: 19744
})

In [4]:
device="cuda" if torch.cuda.is_available() else "cpu"
checkpoint="dphn/Dolphin3.0-Llama3.1-8B"
model=AutoModelForCausalLM.from_pretrained(checkpoint, dtype=torch.bfloat16, device_map="auto").to(device)
tokenizer=AutoTokenizer.from_pretrained(checkpoint)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [8]:
system_prompt="You are a paraphrasing agent. You paraphrase the sentences by using maintaining same tone and same level of toxicity"
def generate_new_sentence(ex_sentence):
    prompt=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": ex_sentence}
    ]
    tokenized_prompt=tokenizer.apply_chat_template(prompt, add_generation_prompt=True, tokenize=True, return_tensors="pt").to(device)
    input_length=tokenized_prompt.shape[1]
    attention_mask = torch.ones_like(tokenized_prompt)
    output_tensors=model.generate(tokenized_prompt,
                                  attention_mask=attention_mask,
                                  pad_token_id=tokenizer.eos_token_id)[0][input_length:]
    return tokenizer.decode(output_tensors, skip_special_tokens=True)

In [9]:
def generate_new_samples(sample_ds):
    lst=[]
    for data in tqdm(sample_ds, desc="Generating new samples."):
        dct={}
        for k,v in data.items():
            dct[k]=generate_new_sentence(v)
        lst.append(dct)
    return lst

In [ ]:
new_samples=generate_new_samples(sample_ds)

Generating new samples.:   0%|          | 0/19744 [00:00<?, ?it/s]

In [ ]:
pprint(new_samples)

In [ ]:
df=pd.DataFrame(new_samples)

In [ ]:
ds=Dataset.from_pandas(df, preserve_index=False)
ds
ds=concatenate_datasets([sample_ds, ds])
ds

In [ ]:
ds.push_to_hub("harsha2946/ParaDetox-Augmented")

# ASAD

In [3]:
sample_ds=load_dataset("s-nlp/paradetox", split="train[:10]")
sample_ds

Dataset({
    features: ['en_toxic_comment', 'en_neutral_comment'],
    num_rows: 10
})

In [4]:
device="cuda" if torch.cuda.is_available() else "cpu"
checkpoint="dphn/Dolphin3.0-Llama3.1-8B"
model=AutoModelForCausalLM.from_pretrained(checkpoint, dtype=torch.bfloat16, device_map="auto").to(device)
tokenizer=AutoTokenizer.from_pretrained(checkpoint)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [5]:
system_prompt="You are a paraphrasing agent. You paraphrase the sentences by using maintaining same tone and same level of toxicity"
def generate_new_sentence(ex_sentence):
    prompt=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": ex_sentence}
    ]
    tokenized_prompt=tokenizer.apply_chat_template(prompt, add_generation_prompt=True, tokenize=True, return_tensors="pt").to(device)
    input_length=tokenized_prompt.shape[1]
    attention_mask = torch.ones_like(tokenized_prompt)
    output_tensors=model.generate(tokenized_prompt,
                                  attention_mask=attention_mask,
                                  pad_token_id=tokenizer.eos_token_id)[0][input_length:]
    return tokenizer.decode(output_tensors, skip_special_tokens=True)

In [6]:
def generate_new_samples(sample_ds):
    lst=[]
    for data in tqdm(sample_ds, desc="Generating new samples."):
        dct={}
        for k,v in data.items():
            dct[k]=generate_new_sentence(v)
        lst.append(dct)
    return lst

In [7]:
new_samples=generate_new_samples(sample_ds)

Generating new samples.:   0%|          | 0/10 [00:00<?, ?it/s]

In [8]:
new_samples

[{'en_toxic_comment': 'He also possessed steel balls!',
  'en_neutral_comment': 'He was courageous as well!'},
 {'en_toxic_comment': 'That guy would have fit in perfectly at the API, surrounded by all the other misfits and trouble',
  'en_neutral_comment': 'It would have been beneficial if he attended API. He would have fit in perfectly.'},
 {'en_toxic_comment': "I'm not going to sell the damn picture, I just want to contribute to the damn article.",
  'en_neutral_comment': "I'm not planning on selling the picture; I just want to contribute to the article."},
 {'en_toxic_comment': 'The content produced by CNN and other news outlets is absolutely disgraceful.',
  'en_neutral_comment': 'The news being produced by CNN and other news agencies is absolutely outrageous.'},
 {'en_toxic_comment': "They don't exist because neither is a pathological liar like Trump.",
  'en_neutral_comment': "The explanation for their absence is that they don't lie like Trump."},
 {'en_toxic_comment': 'I hope th